Copyright © 2020 IUBH Internationale Hochschule

### galton brett 


**Konzept:** (https://de.wikipedia.org/wiki/Galtonbrett):

"Das Galtonbrett besteht aus einer regelmäßigen Anordnung von Hindernissen, an denen eine von oben eingeworfene Kugel jeweils nach links oder rechts abprallen kann. Nach dem Passieren der Hindernisse werden die Kugeln in Fächern aufgefangen, um dort gezählt zu werden." 
"... Ein grundlegendes mathematisches Gesetz, der zentrale Grenzwertsatz, garantiert, dass eine nahezu beliebig zusammengesetzte Verteilung solcher sehr kleinen und sehr zahlreichen Einzelstörungen in der Summe gegen die glockenförmige gaußsche Normalverteilung konvergiert. ... Bei einer endlichen Zahl von Störungen, wie beim Galtonbrett, erhält man die Binomialverteilung, die im Grenzwert vieler Störungen und vieler Fächer ebenfalls gegen die Normalverteilung konvergiert, siehe den Grenzwertsatz von Moivre-Laplace."

**Simulation:**
Mit den Klassen 'board', 'kugel' und 'galton-board' kann eine Simulation des Galton-Boards gestartet werden. 
Das 'board' zeichnet die Behälter und die entsprechenden Hindernisse in Dreiecksform. Jedes Kugel-objekt enthält eine zufällige Bernoulli-Kette über der der Weg durch die Hindernisse bestimmt ist. Der Weg rechts am Hinderniss vorbei entspricht 1, der Weg links am Hindernis vorbei entspricht dem Wert 0.
Die Länge der Kette wird über die Anzahl der Slots bestimmt: lKette = nSlots -1
Die Anzahl der Hindernisreihen entsprechen der Länge der bernoulli-kette.
 
**Parameter**

- nSlots = Anzahl der Behälter        (default = 10)
- nBalls = Anzahl der Kugeln          (default = 100)
- nInterval = Intervalgeschwindigkeit (default = 1 ms)


In [1]:
# %matplotlib notebook
%matplotlib widget

import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.stats import bernoulli, norm


In [2]:
class board:
    def __init__(self, nSlots, ax, y0, xInterval, yInterval):
        self.ax = ax
        self.nSlots = nSlots
        self.res = np.zeros(nSlots)
        self.lenChain = nSlots - 1
        self.indexM = np.zeros([self.lenChain, (self.lenChain * 2 - 1)])
        #
        self.markerSize = 5
        self.xInterval = xInterval
        self.yInterval = yInterval
        self.xOffset = 5
        self.y_0 = y0
        #
        #
        self.xLim = [0, (self.xOffset + self.lenChain + self.xOffset -1)]
        self.yLim = [0, (self.y_0 * 1.20)]
        self.xm = 5
    #--------------------------------------------------------------------
    def initBoard(self):
        """ 
        Matrix für Dreieck bilden:
        die gleichmäßig verteilten Hindernisse bilden ein Dreieck.
        Die Matrix hat folgende Größe: länge der Kette x (2* länge der Kette -1), [2x3, 3x5, 4x7] usw.
        
                    ***+***  -> Matrix zeilenweise füllen. 
                    **+*+**  -> Mit letzter Zeile beginnen und index-Vektor 
                    *+*+*+*  -> Zeile für Zeile verkleinern
                    +*+*+*+
        
        mit Hilfe dieser Matrix werden die Hindernisse als kreisförmige Markierungen in die Figur gezeichnet
        """
        rowIndex = np.arange(0, self.indexM.shape[-1], 2)
        for i in list(range((self.lenChain - 1), -1, -1)):
            if i == (len(self.indexM)-1):
                self.indexM[i][rowIndex] = 1
                continue
            rowIndex = rowIndex[0:-1] +1
            self.indexM[i][rowIndex] = 1
        #
        # Markierungen zeichnen
        #
        x = self.xOffset
        y = self.y_0
        for i in range(0, len(self.indexM)):
            x = self.xOffset
            for c in range(0, self.indexM.shape[-1]):
                if self.indexM[i][c]:
                    self.ax.plot(x, y, 'ok', markersize=str(self.markerSize))
                x += self.xInterval
            y -= self.yInterval

        self.ax.set_xlim(self.xLim)
        self.ax.set_ylim(self.yLim)
        #
        # vertikale Linien um die Behälter anzudeuten
#         ymax = 2
#         self.ax.vlines(x=(self.xOffset - self.xInterval), ymin=0, ymax= ymax, ls="-", lw=3)
#         self.ax.vlines(x=(self.xOffset + self.lenChain + self.xInterval - 1), ymin=0, ymax= ymax, ls="-", lw=3)
#         vx = np.arange(self.xOffset - 1, self.xOffset + self.lenChain, 1)
#         self.ax.vlines(x=vx, ymin=0, ymax =ymax, ls="-", lw=3)
        #
        # Mitte der Figur:
        self.xm = (self.ax.get_xlim()[-1] / 2)
        # No ticks
        self.ax.set_xticks([])
        self.ax.set_yticks([])


In [3]:
class Kugel:
    """
    Klasse 'Kugel':  
    Ein galton brett enthält x Kugeln, die zufällig durch die Hindernisse fallen und am Ende
    in einem Behälter landen. Der Weg der Kugeln wird über eine zufällige Bernoulli-Reihe vorgegeben. Die Kugeln werden
    nacheinander durch das Brett geführt.

    Member:
    - line: lin2D objekt das die Kugel im Diagram representiert
    - gltWay: Bernoulli-Reihe mit der Länge nSlots-1
    - runIndex: inkremenntierender Index, der zur Steuerung durch das Brett entlang der bernoulli-Kette verwendet wird.
        
    Methoden:
    - move_right(): Kugel nach rechts bewegen
    - move_left(): Kugel nach links bewegen
    """
    def __init__(self, ax, gltWay, y0, xInterval, yInterval):
        self.x = (ax.get_xlim()[-1] / 2)
        self.y = y0 + yInterval
        self.xInterval = xInterval
        self.yInterval = yInterval
        self.line, = ax.plot(self.x, self.y, 'or', markersize="8")
        self.color = 'red'
        self.gltWay = gltWay
        self.gltLen = len(gltWay)
        self.runIndex = 0
    #
    def move_right(self):
        if isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0][0]
            self.y = self.line.get_data()[-1][0]
            self.line.set_data(self.x + self.xInterval, self.y - 2)
            return
        if not isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0]
            self.y = self.line.get_data()[-1]
            self.line.set_data(self.x + self.xInterval, self.y - self.yInterval)
        return
    #
    def move_left(self):
        if isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0][0]
            self.y = self.line.get_data()[-1][0]
            self.line.set_data(self.x - self.xInterval, self.y - 2)
            return
        if not isinstance(self.line.get_data()[0], np.ndarray):
            self.x = self.line.get_data()[0]
            self.y = self.line.get_data()[-1]
            self.line.set_data(self.x - self.xInterval, self.y - self.yInterval)
        return

In [4]:
#-----------------------------------------------------------------------------------
class GaltonBrett:
    """ 
    Klasse GaltonBrett: Übergeordnete Klasse um die Steuerung der Animation übersichtlicher zu gestalten.
                        Klasse enthältt als Member das Board und die Kugeln.
    
    Member:
    - board:      board-objekt 
    - balls:      Kugel-objekt
    - activeBall: index zur Kugel die aktuell durchs Brett geführt wird
    - ax:         ax-objekt des Diagrams
    - ax2:        2. ax-objekt für 2.y-Achse. Bezug für Ergebnisfitting. 
    
    Methoden:
    - getBall(iBall): einzelne Kugel zurückgeben
    - getBallsCount(): Anzahl aller Kugeln zurückgeben
    - getSlotsCount(): Anzahl der Behälter zurückgeben
    - getAxis():       ax-Objekt der Figur zurückgeben
    - drawResult():    Gesamtergebnis als bar-plot anzeigen
    - clearBall(iBall):    Einzelne Kugel aus dem Diagramm entfernen
    
    """ 
    def __init__(self, ax, nSlots, nBalls):
        self.balls = []
        self.activeBall = 0
        self.ax = ax
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        self.text = ax.text(0.05, 0.15,'0', transform=self.ax.transAxes, fontsize=10,verticalalignment='top', bbox=props)
        self.ax2 = self.ax.twinx()
        self.ax2.set_yticks([])
        self.xInterval = 0.5
        self.yInterval = 1.5
        #
        if nSlots > 10:
            y0 = 15 * self.yInterval
        else:
            y0 = 15       
        #
        if nSlots > 10 or nBalls > 20:
            y0 = 15 + 0.30*nBalls
        #
        self.board = board(nSlots, ax, y0, self.xInterval, self.yInterval)
        #
        # Figur initialisieren:
        self.board.initBoard()
        #
        # Kugeln in liste speichern
        lenWay = nSlots-1
        self.result = []
        for i in range(nBalls):
            self.balls.append(Kugel(ax, bernoulli.rvs(0.5, size = lenWay), self.board.y_0, self.board.xInterval, self.board.yInterval))
            self.result.append(np.sum(self.balls[i].gltWay))
    #
    def getBall(self, index):
        return self.balls[index]
    #
    def getBallsCount(self):
        return len(self.balls)
    #
    def getSlotsCount(self):
        return self.board.nSlots
    #
    def getAxis(self):
        return self.board.ax
    #
    def clearBall(self, iBall):
        self.balls[iBall].line.remove()
    #
    def AddBallResult(self, iResult):
        self.board.res[iResult] += 1
        iMax = self.board.lenChain + self.board.xOffset
        x = np.arange(self.board.xOffset -1, iMax, 1)
        self.rects = self.board.ax.bar(x, self.board.res , color="blue")
#         self.board.ax.bar(self.board.xOffset - self.board.xInterval, self.board.res[0], color="blue" )
#         self.board.ax.bar(x, self.board.res[1:len(x)+1] , color="blue")
#         self.board.ax.bar(iMax - self.board.xInterval, self.board.res[-1],color="blue")
        #
    #
    def DrawFittingFunction(self, x, y):
        # draw fitting with a second y-axis
        #
        # höhe der einzelnen behälter anzeigen:
        #
        for rect in self.rects:
            height = rect.get_height()
            self.board.ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,'%d' % int(height),ha='center', va='bottom')
        #
        self.ax2.plot(x, y, 'C1')        
        self.ax2.set_ylim(0, np.amax(y) * 1.5)
#-----------------------------------------------------------------------------------

In [5]:
def run_galton(*args):
    """
    update-funktion der Animation
    - für jede Kugel durch die Hindernisse und speichert das Ergebnis als bar-plot
    - beendet die Animation wenn alle Kugeln durch die Hindernisse gelaufen sind und
      fittet eine Normalverteilung aus dem Ergebnis
    """
    #
    # Ende der Animation wenn alle Kugeln durch die Hindernisse gelaufen sind
    if galton.activeBall >= galton.getBallsCount(): 
        #
        # Ergebnis (behälter-Inhalte) mit einer Normalverteilung fitten
        #
        # - die ermittelte Normalverteilung wird in die Mitte des boards gelegt.
        #
#         mu, sigma = norm.fit(np.where(galton.board.res > 0))
#         print("fitting it board.res > 0:\tmu: {0:.2f}\tsigma: {1:.2f}".format(mu, sigma))
        #
        fitting = norm.fit(galton.result)
#         print("fitting mit galton.result:\tmu: {0:.2f}\tsigma: {1:.2f}".format(fitting[0], fitting[1]))
        
        xFit =  np.arange(galton.ax.get_xlim()[0], galton.ax.get_xlim()[-1], 1/200)
        # wenn mu eine ganze Zahl ist, wird die Normalverteilung in der Mitte angezeigt, andernfalls +/- xIntervall versetzt.
        #
        if (galton.board.nSlots/2 - fitting[0]) == 0:
            pdfFit = norm.pdf(xFit, galton.board.xm, fitting[1])
        #
        if (galton.board.nSlots/2 - fitting[0]) > 0:
            pdfFit = norm.pdf(xFit, galton.board.xm - galton.board.xInterval, fitting[1])
        #
        if (galton.board.nSlots/2 - fitting[0]) < 0:
            pdfFit = norm.pdf(xFit, galton.board.xm + galton.board.xInterval, fitting[1])
        #
        #
        textstr = '\n'.join((\
                             "Parameter der Normalverteilung",
                             r'$\mu=%.2f$' % (fitting[0], ),
                             r'$\mathrm{\sigma}=%.2f$' % (fitting[1], ),
                             r'$\mathrm{max =}%.2f$' %(np.amax(pdfFit), )
                             ))
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        galton.ax.text(0.05, 0.95, textstr, transform=galton.ax.transAxes, fontsize=10,\
                       verticalalignment='top', bbox=props)
        #
        galton.DrawFittingFunction(xFit, pdfFit)
        #
#         galton.DrawFittingFunction(xFit, norm.pdf(xFit, fitting[0], fitting[1]))
        #
        anim.event_source.stop()
        return galton
        #
    #
    # Kugelobjekt holen, das gerade bewegt werden soll
    ActBall = galton.getBall(galton.activeBall)
    #
    # is die aktuelle Kugel bereits am Ende angelangt?
    if ActBall.runIndex >= (ActBall.gltLen - 1):
    #
        # Behälterinhalt um 1 erhöhen. 
        galton.AddBallResult(np.sum(ActBall.gltWay))
        # Kugel aus Diagram löschen
        galton.clearBall(galton.activeBall)
        # Kugelzähler um 1 erhöhen
        galton.activeBall += 1
        return ActBall.line
    #
    # Kugel am Hindernis links/rechts bewegen
    if ActBall.runIndex < ActBall.gltLen:
        if ActBall.gltWay[ActBall.runIndex] > 0:
            ActBall.move_right()
        #
        if ActBall.gltWay[ActBall.runIndex] == 0:
            ActBall.move_left()
        #
    ActBall.runIndex += 1
    galton.text.set_text(r'$\Sigma = %d$' %(galton.activeBall +1, ))

    
    return ActBall.line


In [8]:
# Parameter der Simulation
#
nSlots = 13
nBalls = 50
nInterval = 100
#

In [9]:
# Animation mit obigen Parametern starten
# 
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
#
galton = GaltonBrett(axes, nSlots, nBalls)
#
anim = FuncAnimation(fig, run_galton, interval=nInterval, blit=True)
#
# historgram and gaussian-fit
#
# axes[1].hist(galton.result, bins=nSlots, density=True, label='experiment')
#
fitting = norm.fit(galton.result)
xLim = axes.get_xlim()
x2 = np.arange(xLim[0], xLim[-1], 1/200)
# axes[1].plot(x2, norm.pdf(x2, fitting[0], fitting[1]), label='gaussian fit')
#
# axes[1].set_xticks([])
# axes[1].set_yticks([])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Copyright © 2020 IUBH Internationale Hochschule